# **Semiparametric SVM training using subgradients in Spark **

#### bla, bla, bla. 

#### We will benchmark the algorithms with data files from UCI:

* **Ripley**, the Ripley dataset
* **Kwok**, the Kwok dataset
* **Twonorm**, the Twonorm dataset
* **Waveform**, the Waveform dataset
* **Covertype**, the Covertype dataset


In [ ]:
# definir variable de usuario y ejecutar condicionalmente cualquier código que dependa del contexto de ejecución
# no borrar código, simplemente ejecutarlo o no con un "if"
# cada uno mantiene actualizada su parte del "if" y no toca la del otro

user = 'navia'
#user = 'roberto'
modeloSGMA_IRWLS = True

#modelo = 'hybrid' 
#modelo = 'kernelgrad' 
overwrite_results = False # overwrites results even when the result file exists, skips the execution otherwise

if user == 'roberto':
    # definir sc
    import findspark
    findspark.init()
    from pyspark import SparkConf, SparkContext
    conf = (SparkConf().setMaster("local[4]").setAppName("My app").set("spark.executor.memory", "2g"))
    sc = SparkContext(conf = conf)
    import common.lib.svm_utils as SVM_UTILS
    import common.lib.quadtree_utils as QUADTREE
    import numpy as np
    from pyspark.mllib.regression import LabeledPoint
    import pickle
    from math import sqrt
    %matplotlib inline
    # Se importan las funciones
    from common.lib.SGMAUtils import SGMA
    from common.lib.IRWLSUtils import IRWLS
    from common.lib.ResultsUtils import show_results



if user == 'navia':
    sc.addPyFile("file:///export/usuarios01/navia/spark/SVM_spark/common/lib/svm_utils.py")
    sc.addPyFile("file:///export/usuarios01/navia/spark/SVM_spark/common/lib/IRWLSUtils.py")   
    sc.addPyFile("file:///export/usuarios01/navia/spark/SVM_spark/common/lib/KernelUtils.py")   
    sc.addPyFile("file:///export/usuarios01/navia/spark/SVM_spark/common/lib/ResultsUtils.py")   
    sc.addPyFile("file:///export/usuarios01/navia/spark/SVM_spark/common/lib/SGMAUtils.py")   
    import svm_utils as SVM_UTILS
    from SGMAUtils import SGMA
    from IRWLSUtils import IRWLS
    from ResultsUtils import show_results
    import numpy as np
    from pyspark.mllib.regression import LabeledPoint
    import pickle
    from math import sqrt
    %matplotlib inline

    
if modeloSGMA_IRWLS:

    # Esta función habrá que moverla a una librería.
    # mllib tiene una función que hace esto pero por alguna razón luego va todo muy lento.
    
    from sklearn.datasets import load_svmlight_file
    from pyspark.mllib.regression import LabeledPoint   
    
    def loadFile(filename,sc,dimensions):
        X,Y = load_svmlight_file(filename,dimensions)
        X=X.toarray()
        return sc.parallelize(np.concatenate((Y.reshape((len(Y),1)),X),axis=1)).map(lambda x: LabeledPoint(x[0],x[1:]),12)
    
    # Se carga entrenamiento y test
    XtrRDD = loadFile('data/a9a',sc,123)
    XtstRDD = loadFile('data/a9a.t',sc,123)
    
    # Se declaran las variables
    datasetSize = XtrRDD.count()
    NC = 50
    sigma = np.sqrt(123)
    gamma = 1.0/(sigma*sigma)
    C = 1000
    samplingRate=min(1.0,1000.0/datasetSize)    
    
    # Se ejecuta el algoritmo
    Bases = SGMA(XtrRDD,NC,gamma,samplingRate)
    Pesos = IRWLS(XtrRDD,Bases,C,gamma)
    
    show_results(XtstRDD,Bases,Pesos,gamma)     
    

kdatasets = [1, 2, 3, 4]
folds = [1, 2, 3, 4, 5]

#Niter = 2
Nnodes = 32
Samplefraction = 0.05

Niter = 150
NCs = [5, 10, 25, 50, 100, 200]

#Niter = 300
#NCs = [50, 100, 200]


kdatasets = [1, 2, 3, 4]
folds = [1, 2, 3, 4, 5]
Niters = [50, 100, 200]
NCs = [5, 10, 25, 50, 100, 200]
modelos = ['hybrid', 'kernelgrad']
#modelos = ['kernelgrad']

for modelo in modelos:
    for kdataset in kdatasets:
        for kfold in folds:
            for Niter in Niters:
                x_tr, y_tr, x_val, y_val, x_tst, y_tst, name_dataset = SVM_UTILS.load_data(kdataset, kfold)
                NI = x_tr.shape[1]
                sigma = sqrt(NI)
                C = 10.0
                XtrRDD = sc.parallelize(np.hstack((y_tr, x_tr)), Nnodes).map(lambda x: LabeledPoint(x[0], x[1:len(x)]))
                XvalRDD = sc.parallelize(np.hstack((y_val, x_val)), Nnodes).map(lambda x: LabeledPoint(x[0], x[1:len(x)]))
                XtstRDD = sc.parallelize(np.hstack((y_tst, x_tst)), Nnodes).map(lambda x: LabeledPoint(x[0], x[1:len(x)]))
                for NC in NCs:
                    print "Dataset = %s, modelo = %s, kfold = %d, Niter = %d, NC = %d" % (name_dataset, modelo, kfold, Niter, NC)
                    filename = './results/dataset_' + str(kdataset) + '_modelo_' + modelo + '_NC_' + str(NC) + '_Niter_' + str(Niter) + '_kfold_' + str(kfold) + '.pkl'
                    try:
                        f = open(filename,'r')
                        f.close()
                        file_exists = True
                    except:
                        file_exists = False
                        pass
                    execute = False
                    if file_exists:
                        if overwrite_results:
                            execute = True
                    else:
                        execute = True                                  
                    if execute:
                        
                        if modelo == 'hybrid':
                            auc_val, auc_tst, exe_time = SVM_UTILS.train_hybridSVM(XtrRDD, XvalRDD, XtstRDD, sigma, C, NC, name_dataset, Niter, Samplefraction)
                        
                        if modelo == 'kernelgrad':
                            auc_val, auc_tst, exe_time = SVM_UTILS.train_kernelgrad(XtrRDD, XvalRDD, XtstRDD, sigma, C, NC, name_dataset, Niter, Samplefraction)
                        
                        with open(filename, 'w') as f:
                            pickle.dump([auc_val, auc_tst, exe_time], f)
